In [2]:
import json

import requests

import pprint

In [2]:
releases_url = "https://data.neonscience.org/api/v0/releases"
releases = requests.get(releases_url).json()
# json.dump(releases, open("releases.json", "w"), indent=2)

In [3]:
release_names = [release['release'] for release in releases['data']]

In [4]:
release_names

['RELEASE-2021', 'RELEASE-2022', 'RELEASE-2023']

In [5]:
products_url = "https://data.neonscience.org/api/v0/products"
products = requests.get(products_url).json()
# json.dump(products, open("products.json", "w"), indent=2)

In [6]:
product_long_code_to_description = {i['productCodeLong']:i['productDescription'] for i in products['data']}
# json.dump(product_long_code_to_description, open("product_long_code_to_description.json", "w"), indent=2)

In [7]:
metagenomics_product_long_code_to_description = {i['productCodeLong']:i['productDescription'] for i in products['data'] if "metagenomic" in i['productDescription'].lower()}
# json.dump(metagenomics_product_long_code_to_description, open("metagenomics_product_long_code_to_description.json", "w"), indent=2)


In [8]:
metagenomics_product_long_code_to_description

{'NEON.DOM.SITE.DP1.10107.001': 'Metagenomic sequence data from soil samples',
 'NEON.DOM.SITE.DP1.20279.001': 'Metagenomic sequence data from benthic samples',
 'NEON.DOM.SITE.DP1.20281.001': 'Metagenomic sequence data from surface water samples'}

In [9]:
sites_by_release_url = "https://data.neonscience.org/api/v0/sites?release=RELEASE-2023"

In [10]:
# if the releases are cumulative, then we wouldn't need to do the outer loop
# productCodeLong vs productCode

released_metagenomics = {}

for i in release_names:
    for k,v in metagenomics_product_long_code_to_description.items():
        significant_product_slices = k.split(".")[3:6]
        # i think this is directly available from some other key
        product_rhs = ".".join(k.split(".")[3:6])
        url = f"https://data.neonscience.org/api/v0/releases/{i}/products/{product_rhs}"
        released_metagenomics[f"{i}:{k}"] = requests.get(url).json()["data"]

In [11]:
# json.dump(released_metagenomics, open("released_metagenomics.json", "w"), indent=2)

In [12]:
metagenomics_data_urls = []

for k,v in released_metagenomics.items():
    if "siteCodes" in v:
        for i in v['siteCodes']:
            if "siteCode" in i:
                if "availableDataUrls" in i:
                    for j in i['availableDataUrls']:
                        metagenomics_data_urls.append(j)


In [13]:
metagenomics_data_urls = list(set(metagenomics_data_urls))
metagenomics_data_urls.sort()

In [14]:
# metagenomics_data_urls

In [15]:
urls = []

for metagenomics_data_url in metagenomics_data_urls:
    print(metagenomics_data_url)
    metagenomics_data = requests.get(metagenomics_data_url).json()
    if "data" in metagenomics_data:
        if "files" in metagenomics_data['data']:
            for file in metagenomics_data['data']['files']:
                if "url" in file:
                    url = file['url']
                    urls.append(url)


https://data.neonscience.org/api/v0/releases/RELEASE-2021/data/DP1.10107.001/ABBY/2017-06
https://data.neonscience.org/api/v0/releases/RELEASE-2021/data/DP1.10107.001/BARR/2017-08
https://data.neonscience.org/api/v0/releases/RELEASE-2021/data/DP1.10107.001/BART/2014-05
https://data.neonscience.org/api/v0/releases/RELEASE-2021/data/DP1.10107.001/BART/2014-06
https://data.neonscience.org/api/v0/releases/RELEASE-2021/data/DP1.10107.001/BART/2014-07
https://data.neonscience.org/api/v0/releases/RELEASE-2021/data/DP1.10107.001/BART/2014-08
https://data.neonscience.org/api/v0/releases/RELEASE-2021/data/DP1.10107.001/BART/2016-07
https://data.neonscience.org/api/v0/releases/RELEASE-2021/data/DP1.10107.001/BART/2017-06
https://data.neonscience.org/api/v0/releases/RELEASE-2021/data/DP1.10107.001/BLAN/2017-07
https://data.neonscience.org/api/v0/releases/RELEASE-2021/data/DP1.10107.001/BONA/2018-09
https://data.neonscience.org/api/v0/releases/RELEASE-2021/data/DP1.10107.001/CLBJ/2016-08
https://da

In [16]:
urls = list(set(urls))
urls.sort()

In [17]:
with open('neon_metagenomics_data_urls.txt', 'w') as f:
    for line in urls:
        f.write(f"{line}\n")

In [17]:
###

In [3]:
bart_url = "https://data.neonscience.org/api/v0/sites/BART"
bart_data = requests.get(bart_url).json()
bart_products = bart_data['data']['dataProducts']

In [13]:
bart_product_highlights = {d['dataProductTitle']: d['dataProductCode'] for d in bart_products}

In [16]:
sorted(bart_product_highlights.items())

[('2D wind speed and direction', 'DP1.00001.001'),
 ('Albedo - spectrometer - flightline', 'DP2.30011.001'),
 ('Albedo - spectrometer - mosaic', 'DP3.30011.001'),
 ('Barometric pressure', 'DP1.00004.001'),
 ('Breeding landbird point counts', 'DP1.10003.001'),
 ('Bundled data products - eddy covariance', 'DP4.00200.001'),
 ('Canopy water indices - flightline', 'DP2.30019.001'),
 ('Canopy water indices - mosaic', 'DP3.30019.001'),
 ('Coarse downed wood bulk density sampling', 'DP1.10014.001'),
 ('Coarse downed wood log survey', 'DP1.10010.001'),
 ('Digital hemispheric photos of plot vegetation', 'DP1.10017.001'),
 ('Discrete return LiDAR point cloud', 'DP1.30003.001'),
 ('Ecosystem structure', 'DP3.30015.001'),
 ('Elevation - LiDAR', 'DP3.30024.001'),
 ('Ground beetle sequences DNA barcode', 'DP1.10020.001'),
 ('Ground beetles sampled from pitfall traps', 'DP1.10022.001'),
 ('Herbaceous clip harvest', 'DP1.10023.001'),
 ('High-resolution orthorectified camera imagery', 'DP1.30010.001'),
